In [127]:
import pandas as pd

In [128]:
df_web_data = pd.read_csv('../02_DATA_LIMPIO/df_web_data_combinado_limpio.csv')
df_expanded = pd.read_csv('../02_DATA_LIMPIO/df_final_expanded_limpio.csv')
df_clean = pd.read_csv('../02_DATA_LIMPIO/df_final_demo_limpio.csv')
df_experiment = pd.read_csv('../CSV/df_final_experiment_clients.csv')


In [129]:
df_experiment

,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control
...,...,...
70604,2443347,NaN
70605,8788427,NaN
70606,266828,NaN
70607,1266421,NaN


In [130]:
df_expanded

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation,total_tenure_months
0,836976,6,73,60,U,2,45105.30,6,9,test,145
1,2304905,7,94,58,U,2,110860.30,6,9,control,178
2,1439522,5,64,32,U,2,52467.79,6,9,test,124
3,1562045,16,198,49,M,2,67454.65,3,6,test,390
4,5126305,12,145,33,F,2,103671.75,0,3,control,289
...,...,...,...,...,...,...,...,...,...,...,...
50482,1780858,21,262,68,M,3,372100.59,6,9,test,514
50483,6967120,21,260,68,M,3,4279873.38,6,9,control,512
50484,5826160,20,249,56,F,2,44837.16,2,5,test,489
50485,8739285,19,229,69,F,2,44994.24,1,4,test,457


In [131]:
df_web_data

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
755400,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10
755401,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29
755402,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51
755403,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34


In [132]:
df_web_data['date_time'] = pd.to_datetime(df_web_data['date_time'])

In [133]:
# Combino los DataFrames usando 'client_id' como clave
df_combined = df_web_data.merge(
    df_expanded[['client_id', 'Variation']], 
    on='client_id', 
    how='left'
)

In [134]:
# Ordeno por client_id y date_time para cálculos precisos
df_combined.sort_values(by=['client_id', 'date_time'], inplace=True)

In [135]:
# Creo una columna para calcular el tiempo entre pasos por cliente
df_combined['time_diff'] = df_combined.groupby('client_id')['date_time'].diff().dt.total_seconds()

In [136]:
#DF para resultados por cliente
summary = df_combined.groupby('client_id').agg(
    variation=('Variation', 'first')  # Asumimos que 'variation' no cambia por cliente
).reset_index()

In [137]:
# Conteo cuántas veces realiza cada paso
steps_count = df_combined.pivot_table(
    index='client_id', 
    columns='process_step', 
    values='visit_id',  # Usamos visit_id para el conteo
    aggfunc='count', 
    fill_value=0
)


In [138]:
# Cambio el nombre de las columnas de conteo de pasos
steps_count.columns = [f'step_count_{col}' for col in steps_count.columns]


In [139]:
# Calcular el tiempo total en cada paso
steps_time = df_combined.pivot_table(
    index='client_id', 
    columns='process_step', 
    values='time_diff',  # Usamos time_diff para sumar tiempos
    aggfunc='sum', 
    fill_value=0
)

In [140]:
# Nombramos columnas de tiempo en pasos
steps_time.columns = [f'step_time_{col}' for col in steps_time.columns]

In [141]:
# Calculo cuántas veces cada cliente vuelve a iniciar el proceso 'start'
start_count = df_combined[df_combined['process_step'] == 'start'].groupby('client_id').size()
start_count = start_count.rename('start_count').reset_index()

In [142]:
# Merge de todo en un solo DataFrame
result = summary.merge(steps_count, on='client_id', how='left')
result = result.merge(steps_time, on='client_id', how='left')
result = result.merge(start_count, on='client_id', how='left')

In [143]:
# Rellenar valores NaN en start_count con 0
result['start_count'] = result['start_count'].fillna(0).astype(int)

In [144]:
# Filtro el DataFrame para eliminar filas con valores faltantes en 'variation'
result = result.dropna(subset=['variation'])


In [145]:
result

,client_id,variation,step_count_confirm,step_count_start,step_count_step_1,step_count_step_2,step_count_step_3,step_time_confirm,step_time_start,step_time_step_1,step_time_step_2,step_time_step_3,start_count
3,555,test,1,1,1,1,1,20,0,7,32,99,1
4,647,test,1,1,1,1,1,163,0,7,18,189,1
9,934,test,0,4,0,0,0,0,142,0,0,0,4
10,1028,control,0,1,5,2,1,0,0,252,11,275,1
12,1104,control,0,2,0,0,0,0,744135,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120146,9999150,test,0,2,1,0,0,0,12,6,0,0,2
120148,9999400,test,1,1,1,1,1,24,0,22,27,46,1
120151,9999626,test,0,1,1,0,0,0,0,8,0,0,1
120152,9999729,test,1,4,3,2,1,21,2859672,102,297,39,4


**Creamos la columna 'atrás' para saber los usuarios que han tenido que volver atrás para hacer un paso.**

In [146]:
import numpy as np


In [147]:
# Creo una columna auxiliar que asigne un número a cada paso en el proceso
step_mapping = {'start': 0, 'step_1': 1, 'step_2': 2, 'step_3': 3, 'confirm': 4}
df_combined['step_number'] = df_combined['process_step'].map(step_mapping)

In [148]:
# Calculo si el usuario retrocede comparando cada paso con el paso anterior para cada cliente
df_combined['is_back'] = df_combined.groupby('client_id')['step_number'].diff().fillna(0) < 0

In [149]:
# Ccontamos las veces que cada cliente retrocede
back_count = df_combined.groupby('client_id')['is_back'].sum().reset_index()
back_count.rename(columns={'is_back': 'Atrás'}, inplace=True)

In [150]:
# Merge del conteo de retrocesos al df result
result = result.merge(back_count, on='client_id', how='left')

In [151]:
# Rellenar valores NaN con 0 (en caso de clientes que no retrocedieron)
result['Atrás'] = result['Atrás'].fillna(0).astype(int)

In [152]:
result

,client_id,variation,step_count_confirm,step_count_start,step_count_step_1,step_count_step_2,step_count_step_3,step_time_confirm,step_time_start,step_time_step_1,step_time_step_2,step_time_step_3,start_count,Atrás
0,555,test,1,1,1,1,1,20,0,7,32,99,1,0
1,647,test,1,1,1,1,1,163,0,7,18,189,1,0
2,934,test,0,4,0,0,0,0,142,0,0,0,4,0
3,1028,control,0,1,5,2,1,0,0,252,11,275,1,2
4,1104,control,0,2,0,0,0,0,744135,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50482,9999150,test,0,2,1,0,0,0,12,6,0,0,2,1
50483,9999400,test,1,1,1,1,1,24,0,22,27,46,1,0
50484,9999626,test,0,1,1,0,0,0,0,8,0,0,1,0
50485,9999729,test,1,4,3,2,1,21,2859672,102,297,39,4,2


**Calculo el tiempo total en el todo el proceso**

In [153]:
# Calculamos el tiempo de inicio y finalización del proceso para cada cliente
process_times = df_combined.groupby('client_id').agg(
    process_start=('date_time', lambda x: x[df_combined.loc[x.index, 'process_step'] == 'start'].min()),
    process_end=('date_time', lambda x: x[df_combined.loc[x.index, 'process_step'] == 'confirm'].max())
).reset_index()

In [154]:
# Calculamos la duración del proceso como una diferencia de tiempo
process_times['process_duration'] = process_times['process_end'] - process_times['process_start']

In [155]:
# Merge de los tiempos calculados al df result
result = result.merge(process_times[['client_id', 'process_duration']], on='client_id', how='left')

In [156]:
result

,client_id,variation,step_count_confirm,step_count_start,step_count_step_1,step_count_step_2,step_count_step_3,step_time_confirm,step_time_start,step_time_step_1,step_time_step_2,step_time_step_3,start_count,Atrás,process_duration
0,555,test,1,1,1,1,1,20,0,7,32,99,1,0,0 days 00:02:38
1,647,test,1,1,1,1,1,163,0,7,18,189,1,0,0 days 00:06:17
2,934,test,0,4,0,0,0,0,142,0,0,0,4,0,NaT
3,1028,control,0,1,5,2,1,0,0,252,11,275,1,2,NaT
4,1104,control,0,2,0,0,0,0,744135,0,0,0,2,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50482,9999150,test,0,2,1,0,0,0,12,6,0,0,2,1,NaT
50483,9999400,test,1,1,1,1,1,24,0,22,27,46,1,0,0 days 00:01:59
50484,9999626,test,0,1,1,0,0,0,0,8,0,0,1,0,NaT
50485,9999729,test,1,4,3,2,1,21,2859672,102,297,39,4,2,33 days 02:28:51


In [157]:
# Intento arreglar los calores NaT que entiendo que son personas que no han completado el proceso
sin_duracion = result[result['process_duration'].isna()]

In [158]:
sin_duracion

,client_id,variation,step_count_confirm,step_count_start,step_count_step_1,step_count_step_2,step_count_step_3,step_time_confirm,step_time_start,step_time_step_1,step_time_step_2,step_time_step_3,start_count,Atrás,process_duration
2,934,test,0,4,0,0,0,0,142,0,0,0,4,0,NaT
3,1028,control,0,1,5,2,1,0,0,252,11,275,1,2,NaT
4,1104,control,0,2,0,0,0,0,744135,0,0,0,2,0,NaT
5,1186,control,0,2,1,1,0,0,7546,11,11,0,2,0,NaT
9,1346,test,0,5,2,1,2,0,425,11,7,52,5,2,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50473,9997391,control,0,1,1,3,3,0,0,49,99,75,1,2,NaT
50479,9998921,control,0,1,0,0,0,0,0,0,0,0,1,0,NaT
50482,9999150,test,0,2,1,0,0,0,12,6,0,0,2,1,NaT
50484,9999626,test,0,1,1,0,0,0,0,8,0,0,1,0,NaT


In [159]:
# Asigno un valor por defecto a process_duration donde haya NaT
result['process_duration'] = result['process_duration'].fillna('0')


In [160]:
result['completed'] = (result['step_count_confirm'] > 0) & (~result['process_duration'].isna())


In [161]:
result

,client_id,variation,step_count_confirm,step_count_start,step_count_step_1,step_count_step_2,step_count_step_3,step_time_confirm,step_time_start,step_time_step_1,step_time_step_2,step_time_step_3,start_count,Atrás,process_duration,completed
0,555,test,1,1,1,1,1,20,0,7,32,99,1,0,0 days 00:02:38,True
1,647,test,1,1,1,1,1,163,0,7,18,189,1,0,0 days 00:06:17,True
2,934,test,0,4,0,0,0,0,142,0,0,0,4,0,0 days 00:00:00,False
3,1028,control,0,1,5,2,1,0,0,252,11,275,1,2,0 days 00:00:00,False
4,1104,control,0,2,0,0,0,0,744135,0,0,0,2,0,0 days 00:00:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50482,9999150,test,0,2,1,0,0,0,12,6,0,0,2,1,0 days 00:00:00,False
50483,9999400,test,1,1,1,1,1,24,0,22,27,46,1,0,0 days 00:01:59,True
50484,9999626,test,0,1,1,0,0,0,0,8,0,0,1,0,0 days 00:00:00,False
50485,9999729,test,1,4,3,2,1,21,2859672,102,297,39,4,2,33 days 02:28:51,True


In [162]:
# Cambio el nombre de columnas que comienzan con 'step_count_' a 'count_' para facilitar su uso
result = result.rename(columns=lambda col: col.replace('step_count_', 'count_') if col.startswith('step_count_') else col)


In [163]:
# Igual con las columnas que comienzan con 'step_time_' a 'time_'
result = result.rename(columns=lambda col: col.replace('step_time_', 'time_') if col.startswith('step_time_') else col)


In [164]:
# Para facilitar la interpretación cambio las columnas time_ a minutos.
time_columns = [col for col in result.columns if col.startswith('time_')]

# Convierto los valores de segundos a minutos.
for col in time_columns:
    result[col] = (result[col] / 60).round(2).astype(float)

In [165]:
result

,client_id,variation,count_confirm,count_start,count_step_1,count_step_2,count_step_3,time_confirm,time_start,time_step_1,time_step_2,time_step_3,start_count,Atrás,process_duration,completed
0,555,test,1,1,1,1,1,0.33,0.00,0.12,0.53,1.65,1,0,0 days 00:02:38,True
1,647,test,1,1,1,1,1,2.72,0.00,0.12,0.30,3.15,1,0,0 days 00:06:17,True
2,934,test,0,4,0,0,0,0.00,2.37,0.00,0.00,0.00,4,0,0 days 00:00:00,False
3,1028,control,0,1,5,2,1,0.00,0.00,4.20,0.18,4.58,1,2,0 days 00:00:00,False
4,1104,control,0,2,0,0,0,0.00,12402.25,0.00,0.00,0.00,2,0,0 days 00:00:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50482,9999150,test,0,2,1,0,0,0.00,0.20,0.10,0.00,0.00,2,1,0 days 00:00:00,False
50483,9999400,test,1,1,1,1,1,0.40,0.00,0.37,0.45,0.77,1,0,0 days 00:01:59,True
50484,9999626,test,0,1,1,0,0,0.00,0.00,0.13,0.00,0.00,1,0,0 days 00:00:00,False
50485,9999729,test,1,4,3,2,1,0.35,47661.20,1.70,4.95,0.65,4,2,33 days 02:28:51,True


In [166]:
result.to_csv('../02_DATA_LIMPIO/df_steps_and_time.csv', index=False)

In [167]:
print(result.dtypes)


client_id                     int64
variation                    object
count_confirm                 int64
count_start                   int64
count_step_1                  int64
count_step_2                  int64
count_step_3                  int64
time_confirm                float64
time_start                  float64
time_step_1                 float64
time_step_2                 float64
time_step_3                 float64
start_count                   int64
Atrás                         int64
process_duration    timedelta64[ns]
completed                      bool
dtype: object


In [168]:
time_columns = [col for col in result.columns if col.startswith('time_')]

In [169]:
# Aseguramos que las columnas time_ estén en formato numérico (float)
for col in time_columns:
    result[col] = pd.to_numeric(result[col], errors='coerce') 

In [171]:
result.to_csv('../02_DATA_LIMPIO/df_steps_and_time.csv', index=False)

In [172]:
print(result.dtypes)

client_id                     int64
variation                    object
count_confirm                 int64
count_start                   int64
count_step_1                  int64
count_step_2                  int64
count_step_3                  int64
time_confirm                float64
time_start                  float64
time_step_1                 float64
time_step_2                 float64
time_step_3                 float64
start_count                   int64
Atrás                         int64
process_duration    timedelta64[ns]
completed                      bool
dtype: object
